# Calculating the Moment 0 Map

## Introduction
Computing the line intensity (moment-0) based on a 3D data cube is a fundamental operation. In this tutorial, we will learn how to generate a 3D mask and then integrate the cube along the spectral axis to obtain a robust moment-0 measurement. This tutorial is aimed at undergraduate and graduate students with a background in astrophysics.

## Learning Objectives
1. Determine the noise of a 3D data cube.
2. Learn how to define a signal mask indicating where signal is detected.
3. Calculate the moment-0 mask using a signal mask

## Background:
### The line emission profile ###

In astronomy, we commonly find line emission profiles, that we can describe by a single Gaussian-shaped spectrum (or a combination thereof) that has the following functional form:
$$
S(v) = A \exp\left[-\frac{(v-v_0)^2}{2\sigma_v^2}\right],
$$
In this formula, $A$ describes the *amplitude*, $v_0$ the *centroid velocity* of the line profile, and $\sigma_v$ the *line width*.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import ndimage

**Define generic spectrum**

In [ ]:
#define Gaussian profile
A = 1
v0 = 0
sigma_v = 1

#Define axis
x_axis=np.linspace(-20,20,200)
delta_v = abs(x_axis[1]-x_axis[0])

#spec: The ideal spectrum
spec = A*np.exp(-(x_axis-v0)**2/sigma_v**2)

# spec_noise: Add noise from normal distribution
sigma_noise = 0.1
spec_noise = spec +np.random.normal(0,sigma_noise, len(x_axis))

# spec_noise_realistic: For real-life detectors, the sensitivity generally decreases towards channels 
# at the edge, thereby increasing the noise. Here we try to immitate that by adding noise of increasing
# magnitude towards either side of the spectrum.
spec_noise_realistic = spec_noise + np.random.normal(0, abs(x_axis/20)**3)

In [ ]:
plt.figure(figsize=(9,3))
ax = plt.subplot(1,3,1)
ax.step(x_axis, spec)
ax.set_ylim(-1,2)
ax.axhline(0, zorder=0, color='k', alpha=.5)
ax.set_title('Gaussian')

ax = plt.subplot(1,3,2)
ax.step(x_axis, spec_noise)
ax.set_ylim(-1,2)
ax.set_yticklabels([])
ax.axhline(0, zorder=0, color='k', alpha=.5)
ax.set_title('Gaussian + noise')

ax = plt.subplot(1,3,3)
ax.step(x_axis, spec_noise_realistic)
ax.set_ylim(-1,2)
ax.set_yticklabels([])
ax.axhline(0, zorder=0, color='k', alpha=.5)
ax.set_title('Gaussian + "realistic" noise')
plt.tight_layout()

### Integrated Intensity -- Moment 0

The most common product is the integrated intensity, or the zeroth moment. This is defined as the integral of $S(v)$, or equivalently the sum over the signal in the observed spectrum:
$$
M_0 = \int_{\mathrm{line}} T(v)\,dv \approx \sum_{i\in\{\mathrm{line}\}} T(v_i)\,\delta v,
$$
where $\delta v$ is the channel width in the cube. To improve the S/N, we define a mask that includes channels where we expect line emission (here defined as $\{\mathrm{line}\}$) and only sum over these.

The integrated intensity units is commonly K km/s. However, for this example, we work with arbitrary units.


### Simplest Calculation: Integrating over the whole spectrum

Integrating over the entire spectrum is the easiest way to determine the moment-0. However, since we integrate over the noise, the resulting S/N might be lower.

In [ ]:
moment_0_spec = np.nansum(spec)*delta_v
moment_0_spec_noise = np.nansum(spec_noise)*delta_v
moment_0_spec_noise_real = np.nansum(spec_noise_realistic)*delta_v

print('Moment-0 ideal spectrum:\t\t %.3f [a.u]'%moment_0_spec )
print('Moment-0 simple noise spectrum:\t\t %.3f [a.u]'%moment_0_spec_noise )
print('Moment-0 realistic noise spectrum:\t %.3f [a.u]'%moment_0_spec_noise_real )

**We see that the integrated intensities vary by 10-20\% in the case of large noise. Therefore, defining a mask will make the measurements less sensitive to the noise:**

### Defining a 1D Mask

The mask determination consists of the following steps:

i) Determine the channel noise (rms)

ii) Determine the channels (in 3D the voxels) who's values is above the noise.

iii) Improve the mask by removing spurious false positives.

**Step 1:** Determine the rms

* First we calculate the standard deviation (using the mean absolute deviation, which is more robust to outliers). This will overestimate the noise, since the presence of a line will bias the rms towards larger values.
* We perform a clipping and calculate the rms again. This ensures that the majority of the line emission is not included anymore.

In [ ]:
from astropy.stats import mad_std


rms_spec = mad_std(spec_noise)
rms_spec = mad_std(spec_noise[spec_noise<3*rms_spec])

rms_spec_realnoise = mad_std(spec_noise_realistic)
rms_spec_realnoise = mad_std(spec_noise_realistic[spec_noise_realistic<3*rms_spec_realnoise])

**Comment:** Here we illustrate the procdure for the scenario if we only have one spectrum. However, if you work with numerous spectra (for instance with a 3D data cube), you could calculate the std per channel or group of channels to come up with a noise spectrum. See more details below.

**Plot the spectra and show the calculated rms:**

In [ ]:
plt.figure(figsize=(9,3))
ax = plt.subplot(1,3,1)
ax.step(x_axis, spec)
ax.set_ylim(-1,2)
ax.axhline(0, zorder=0, color='k', alpha=.5)
ax.set_title('Gaussian')

ax = plt.subplot(1,3,2)
ax.step(x_axis, spec_noise)
ax.set_ylim(-1,2)
ax.set_yticklabels([])
ax.axhline(0, zorder=0, color='k', alpha=.5)
ax.set_title('Gaussian + noise')
ax.axhline(rms_spec, color='r', ls=':', label=r'1$\sigma$')
ax.axhline(3*rms_spec, color='r', label=r'3$\sigma$')
ax.legend()

ax = plt.subplot(1,3,3)
ax.step(x_axis, spec_noise_realistic)
ax.set_ylim(-1,2)
ax.set_yticklabels([])
ax.axhline(0, zorder=0, color='k', alpha=.5)
ax.set_title('Gaussian + "realistic" noise')
ax.axhline(rms_spec_realnoise, color='r', ls=':', label=r'1$\sigma$')
ax.axhline(3*rms_spec_realnoise, color='r', label=r'3$\sigma$')
ax.legend()

plt.tight_layout()

**Step 2: Defining a mask**

* We define a high and a low mask at different S/N thresholds
* The high mask (S/N>4) ensures that only real emission is included
* The low mask ensures (S/N>2), that we do not clip away too much emission.
* We then take the high mask and expand it into the low mask

In [ ]:
high_tresh = 4
low_tresh = 2

mask = np.array(spec_noise > high_tresh * rms_spec, dtype = int)
low_mask = np.array(spec_noise > low_tresh * rms_spec, dtype = int)

mask_realistic = np.array(spec_noise_realistic > high_tresh * rms_spec_realnoise, dtype = int)
low_mask_realistic = np.array(spec_noise_realistic > low_tresh * rms_spec_realnoise, dtype = int)

**remove spikes** 

We want to make sure that single spikes above the threshold are not included in our mask. For this, we shift the spectra to the left and right and check if the overlap still is significant per channel (which would indicate that at least three consecutive channels are significantly detected.

In [ ]:
#remove spikes along spectral axis:
mask = np.array((mask + np.roll(mask, 1) + np.roll(mask, -1))>=3, dtype = int)
low_mask = np.array((low_mask + np.roll(low_mask, 1) + np.roll(low_mask, -1))>=3, dtype = int)

mask_realistic = np.array((mask_realistic + np.roll(mask_realistic, 1) + np.roll(mask_realistic, -1))>=3, dtype = int)
low_mask_realistic = np.array((low_mask_realistic + np.roll(low_mask_realistic, 1) + np.roll(low_mask_realistic, -1))>=3, dtype = int)

**Expand channel-wise the high mask into the low mask**

In [ ]:
n_chan=0
while n_chan!=np.nansum(mask):
    n_chan = np.nansum(mask)
    mask = ndimage.binary_dilation(mask).astype(mask.dtype)*low_mask

n_chan=0
while n_chan!=np.nansum(mask_realistic):
    n_chan = np.nansum(mask_realistic)
    mask_realistic = ndimage.binary_dilation(mask_realistic).astype(mask.dtype)*low_mask_realistic

# Further expand the mask a few channels to include the wings
#expand to cover part of edge of the emission line
# adjust n_iter such that you see no missing wings of the spectrum (see plotting below)
n_iter=6
mask = ndimage.binary_dilation(mask, iterations=n_iter).astype(mask.dtype)
mask_realistic = ndimage.binary_dilation(mask_realistic, iterations=n_iter).astype(mask.dtype)

In [ ]:
plt.figure(figsize=(9,3))
ax = plt.subplot(1,3,1)
ax.step(x_axis, spec)
ax.set_ylim(-1,2)
ax.axhline(0, zorder=0, color='k', alpha=.5)
ax.set_title('Gaussian')

ax = plt.subplot(1,3,2)
ax.step(x_axis, spec_noise)
ax.step(x_axis[mask==1], spec_noise[mask==1])
ax.set_ylim(-1,2)
ax.set_yticklabels([])
ax.axhline(0, zorder=0, color='k', alpha=.5)
ax.set_title('Gaussian + noise')
ax.axhline(rms_spec, color='r', ls=':', label=r'1$\sigma$')
ax.axhline(3*rms_spec, color='r', label=r'3$\sigma$')
ax.legend()

ax = plt.subplot(1,3,3)
ax.step(x_axis, spec_noise_realistic)
ax.step(x_axis[mask_realistic==1], spec_noise_realistic[mask_realistic==1])
ax.set_ylim(-1,2)
ax.set_yticklabels([])
ax.axhline(0, zorder=0, color='k', alpha=.5)
ax.set_title('Gaussian + "realistic" noise')
ax.axhline(rms_spec_realnoise, color='r', ls=':', label=r'1$\sigma$')
ax.axhline(3*rms_spec_realnoise, color='r', label=r'3$\sigma$')
ax.legend()

plt.tight_layout()

The orange part of the spectrum shows the significantly detected channels, that we include in the end for the moment-0 calculation. Note that even for the "realistic noise" example, we pick up only the real line emission.

**Step 3: Integrate over the masked channels only**

In [ ]:
moment_0_spec = np.nansum(spec)*delta_v
moment_0_spec_noise = np.nansum(spec_noise*mask)*delta_v
moment_0_spec_noise_real = np.nansum(spec_noise_realistic*mask_realistic)*delta_v

print('Moment-0 ideal spectrum:\t\t %.3f [a.u]'%moment_0_spec )
print('Moment-0 simple noise spectrum:\t\t %.3f [a.u]'%moment_0_spec_noise )
print('Moment-0 realistic noise spectrum:\t %.3f [a.u]'%moment_0_spec_noise_real )

The moment-0 values, in particular for the realistic noise example, do now agree more with the expected value

## Application to a 3D data cube

Here we build on the [radio-astro-tools tutorial](https://github.com/radio-astro-tools/tutorials/blob/master/masking_and_moments/masking_and_moments_guide.ipynb). 

**Step 1: Import the data cube**

In [ ]:
from spectral_cube import SpectralCube
import astropy.units as u

filename = './data/ngc5194_co21.fits'
cube = SpectralCube.read(filename, format='fits')

# Convert to km/s for the spectral unit
cube = cube.with_spectral_unit(u.km / u.s)

**Step 2: Calculate the rms**

In [ ]:
mad_std_map = cube.mad_std(axis=0)
mad_std_map = cube.with_mask(cube<3*mad_std_map).mad_std(axis=0)

In [ ]:
# Make a low and high mask
low_snr_mask = (cube > 2 * mad_std_map).include()
high_snr_mask = (cube > 4 * mad_std_map).include()

#remove spikes along spectral axis
high_snr_mask = high_snr_mask & np.roll(high_snr_mask,1,0)& np.roll(high_snr_mask,-1,0)
low_snr_mask = low_snr_mask & np.roll(low_snr_mask,1,0)& np.roll(low_snr_mask,-1,0)

**Step 3: Remove spatially spurious detections**
Note that this was not necessary for the 1D example, however, now in 3D, we remove spikes by making sure that in 2D, connected regions are larger than a given size.

We will label connected regions in `low_snr_mask` and label them. `low_snr_mask` and `high_snr_mask` are both dask arrays. We define regions as connected as 26-connected, meaning all edge and corner connections are included in 3D. The equivalent in 2D is 8-connected and would have a structure element like:

```
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
```

In [ ]:
import scipy.ndimage as nd

structure = np.ones((3, 3, 3), dtype=bool)

low_snr_mask_labels, num_labels = nd.label(low_snr_mask,
                                           structure=structure)

print(f"Initial number of regions found: {num_labels}")



To remove spurious regions, we will apply 2 criteria:

1. All regions in `low_snr_mask` must contain $>10$ pixels above $4\sigma$ (the `high_snr_mask`). To do this, we calculate the sum of `high_snr_mask` within the `low_snr_mask_labels`.
2. Regions in `low_snr_mask` must contain $>20$ pixels. Similarly, we calculate the sum of `low_snr_mask` within the `low_snr_mask_labels`. 20 pixels in our example is about twice the number of pixels in the beam.

Regions that do not satisfy these criteria are removed from the mask.

In [ ]:
import scipy.ndimage as nd

num_pixels_in_high_snr_mask = nd.sum(high_snr_mask,
                                     labels=low_snr_mask_labels,
                                     index=range(1, num_labels + 1)) # +1 offset for mask labels

# Repeat for the high signal mask.
num_pixels_in_low_snr_mask = nd.sum(low_snr_mask,
                                    labels=low_snr_mask_labels,
                                    index=range(1, num_labels + 1)) # +1 offset for mask labels

# To preserve the low_snr_mask, we will create a new signal mask where we will remove 
# regions that do not pass the criteria.
signal_mask = low_snr_mask

low_min_pixels = 20
high_min_pixels = 10

for num, (high_pix_num, low_pix_num) in enumerate(zip(num_pixels_in_high_snr_mask, num_pixels_in_low_snr_mask)):
    if high_pix_num >= high_min_pixels and low_pix_num >= low_min_pixels:
        # This region passes the criteria. Keep it in the mask.
        continue

    # Remove regions that do not pass the criteria.
    # NOTE: enumerate will start with 0, but the mask labels start at 1
    # We apply a +1 offset to `num` to account for this.
    signal_mask[low_snr_mask_labels == num + 1] = False


In [ ]:
signal_mask_labels, num_labels = nd.label(signal_mask,
                                          structure=structure)

print(f"Final number of regions found: {num_labels}")

Nearly every region we initially found was likely spurious, and far fewer regions remain that are more likely to be "real" CO emission from our source.

Before examining this new signal mask, we can apply an additional step to improve the recovery of faint emission near bright signal. Because we have imposed at $>3\sigma$ criterion in the low signal mask, some faint emission adjacent to the signal will be excluded and we will underestimate properties like the integrated intensity.

One simple way to correct for this is to extend the signal mask immediately around our remaining regions. We can extend the mask by one pixel in each dimension using [binary dilation](https://en.wikipedia.org/wiki/Dilation_%28morphology%29) to superimpose a structure element around every pixel included in the mask. With `dask-image`, we can use `ndmorph.binary_dilation`. 


In [ ]:
# Use a structure that will increase the mask size by 1 pixel in each direction.
# Because we are applying it to the 3D cube, we need its size to be (3, 3, 3):

# In the spatial dimension, expand by 1 pixel in each direction.
# The structure looks like:
# [1, 1, 1]
# [1, 1, 1]
# [1, 1, 1]
structure = np.ones((3, 3), dtype=bool)

# In the spectral dimension, only expand the original pixel along the spectral dimension.
# The structure looks like:
# [0, 0, 0]
# [0, 1, 0]
# [0, 0, 0]

structure_spec = np.zeros((3, 3), dtype=bool)
structure_spec[1, 1] = True

# Add 1 spectral element on each side of the spatial structure.
# np.dstack stacks the arrays along a new 3rd dimension:
structure = np.swapaxes(np.dstack([structure_spec, structure, structure_spec]), 0, 2)

# With scipy.ndimage:
# import scipy.ndimage as nd

signal_mask = nd.binary_dilation(signal_mask, structure=structure, iterations=1)


At this point, we have finished computing the signal mask and now dask can execute the operations, returning a numpy array:


In [ ]:
masked_cube = cube.with_mask(signal_mask)

In [ ]:
peak_intensity_sigmask = masked_cube.max(axis=0)

peak_intensity_sigmask.quicklook()

**Spectral Cube also has the option to easily calculate the moment-0 map**

In [ ]:
masked_moment0 = masked_cube.moment0()

ax = plt.subplot(projection=masked_moment0.wcs)
im = ax.imshow(masked_moment0.value, origin='lower', cmap='inferno')
cbar = plt.colorbar(im)
cbar.set_label('Integrated Intensity (K km/s)')

ax.set_ylabel('Declination')
ax.set_xlabel('Right Ascension')

SpectralCube can also compute other moment maps. See following tutorials for more details:

* https://spectral-cube.readthedocs.io/en/latest/moments.html

* https://radio-astro-tools.github.io/tutorials/